In [3]:
"""
create_gmail_labels.py

One-time (but safe to re-run) script to create Gmail labels
for job application tracking, with this structure:

Job Update/
    Application Incomplete
    Applied
    Assessment
    Interview
    Offer Made
    Other
    Rejected
    Withdrawn
Meet Request
Requires Attention
"""

from __future__ import annotations
import os
from typing import Dict, Iterable

from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError

import sys
from pathlib import Path

# -----------------------------
# Configuration
# -----------------------------
SCOPES = ["https://www.googleapis.com/auth/gmail.settings.basic"]

LABELS = [
    "Job Update/Application Incomplete",
    "Job Update/Applied",
    "Job Update/Assessment",
    "Job Update/Interview",
    "Job Update/Offer Made",
    "Job Update/Other",
    "Job Update/Rejected",
    "Job Update/Withdrawn",
    "Meet Request",
    "Requires Attention",
]


# Getting path of current file's parent directory
path = Path(os.path.dirname(os.getcwd()))
path = str(path)
sys.path.insert(1, path)

# -----------------------------
# Helpers
# -----------------------------

def get_gmail_service():
    creds = Credentials.from_authorized_user_file(f'{path}/config/token.json')
    return build('gmail', 'v1', credentials=creds)


def list_labels(service) -> Dict[str, str]:
    """Return {name: id} of all existing labels."""
    resp = service.users().labels().list(userId="me").execute()
    return {lbl["name"]: lbl["id"] for lbl in resp.get("labels", [])}


def _create_label(service, name: str) -> str:
    """Create one label. Returns label id."""
    body = {
        "name": name,
        "labelListVisibility": "labelShow",
        "messageListVisibility": "show",
    }
    created = service.users().labels().create(userId="me", body=body).execute()
    return created["id"]


def ensure_labels(service, label_names: Iterable[str]) -> Dict[str, str]:
    """Ensure labels exist. Returns {name: id}. Safe to re-run."""
    existing = list_labels(service)
    ensured: Dict[str, str] = {}
    for name in label_names:
        if name in existing:
            ensured[name] = existing[name]
            continue
        try:
            lid = _create_label(service, name)
            ensured[name] = lid
        except HttpError as e:
            if e.resp.status == 409:  # conflict, likely already exists
                existing = list_labels(service)
                ensured[name] = existing[name]
            else:
                raise
    return ensured

# -----------------------------
# Main
# -----------------------------
if __name__ == "__main__":
    service = get_gmail_service()
    created = ensure_labels(service, LABELS)
    print("\nCreated/ensured labels:")
    for name, lid in created.items():
        print(f"{name} -> {lid}")



Created/ensured labels:
Job Update/Application Incomplete -> Label_4552311577949953739
Job Update/Applied -> Label_3701903105754377425
Job Update/Assessment -> Label_6526643710159138362
Job Update/Interview -> Label_1176858450540225884
Job Update/Offer Made -> Label_682296045437184348
Job Update/Other -> Label_2627695902442400621
Job Update/Rejected -> Label_5388285104804611094
Job Update/Withdrawn -> Label_1877962796526733589
Meet Request -> Label_4718307430553739191
Requires Attention -> Label_7278646546916726427
